In [1]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'


In [2]:
#@ generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [3]:
#@ converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [4]:
#@ fitness function  ==== linear_power_vect
def sum_linear_powers(array):
    return np.sum(array, axis=1)

In [5]:
# $ calculate the bit rate matrix
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [6]:
#@
def selection_half_population(size_pop, pop_size_dbm, row_sums):
    L = size_pop // 2
    
    # Combine the population and their corresponding row sums
    help_fun = np.column_stack((pop_size_dbm, row_sums))

    # Sort the combined array in ascending order of row sums
    sorted_help_fun = help_fun[np.argsort(help_fun[:, -1])]
    
    # Select the top half of the population
    parent = sorted_help_fun[:L, :-1]
    parent_EE = sorted_help_fun[:L, -1]
    
    # Combine the selected parents and their corresponding row sums
    selected_parents = [[list(parent_row), parent_EE_value] for parent_row, parent_EE_value in zip(parent, parent_EE)]
    
    return selected_parents

In [7]:
def extract_powers_in_dbm(selected_parents):
    powers = [list(parent[0]) for parent in selected_parents]
    return powers

In [8]:
#@
def one_point_crossover_first_second(parents):
    offspring = []
    
    # Perform crossover between the first two parents
    parent1 = parents[0][0]
    parent2 = parents[1][0]
    crossover_point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    offspring.append(offspring1)
    offspring.append(offspring2)

    # Perform crossover between the second two parents
    parent3 = parents[2][0]
    parent4 = parents[3][0]
    crossover_point = random.randint(1, len(parent3) - 1)
    offspring3 = parent3[:crossover_point] + parent4[crossover_point:]
    offspring4 = parent4[:crossover_point] + parent3[crossover_point:]
    offspring.append(offspring3)
    offspring.append(offspring4)

    return offspring




In [9]:
random.randint(1, 3)

1

In [10]:
def mutation(offspring, powers):
    mutated_offspring = []
    for child in offspring:
        mutated_child = child.copy()
        mutation_position = random.randint(0, len(child) - 1)
        new_gene = random.choice(powers)
        
        while new_gene == mutated_child[mutation_position]:
            new_gene = random.choice(powers)
        
        mutated_child[mutation_position] = new_gene
        mutated_offspring.append(mutated_child)
    return mutated_offspring



In [11]:
def concatenate_parents_and_offspring(selected_parents, mutated_offspring):
    combined_population = []

    # Add selected parents to the combined population
    for parent, parent_EE_value in selected_parents:
        combined_population.append(parent)

    # Add mutated offspring to the combined population
    for offspring in mutated_offspring:
        combined_population.append(np.array(offspring))

    # Convert the combined population list to a NumPy array
    combined_population = np.array(combined_population)

    return combined_population

# # Usage
# combined_population = concatenate_parents_and_offspring(selected_parents, mutated_offspring)
# print("Combined Population:")
# print(combined_population)


In [12]:
powers               = [42,44,46]
num_antennas_BS      = 8
pop_size             = 40

D=10**6                                                            #bandwidth
N=-174 + 10 * math.log10(D)                                        #noise in dbm
linear_noise = calculate_linear_trans_power([N])
         
population = generate_random_solutions(powers, num_antennas_BS, pop_size)

In [13]:
def GA(population):
    Lin_population = calculate_linear_trans_power(population)
    Lin_vector     = sum_linear_powers(Lin_population)
    parents        = selection_half_population(pop_size, population, Lin_vector)
    
    best_power     = parents[0][1] 
    best_scenario  = np.array(parents[0][0]).astype(int)
    
    next_generation= np.array(extract_powers_in_dbm(parents)).astype(int)
       
    crossover       = np.array(one_point_crossover_first_second(parents)).astype(int)
    mutation1       = np.array(mutation(crossover, powers)).astype(int)
    #print("crossover",crossover)
    #print("mutation1",mutation1)
    
    next_generation=  concatenate_parents_and_offspring(parents, mutation1)
    population     = next_generation

    return population,best_power,best_scenario

In [14]:
max_G = 100

for i in range(max_G):
    
    population,best_powerr,best_scenarioo = GA(population)
    print("generation:        ",i+1)
    #print("population",population)
    print("Best_scenario      ",best_scenarioo)
    print("Best_power         ",best_powerr)
    print("************************************************")
   

generation:         1
Best_scenario       [44 42 42 42 44 44 42 44]
Best_power          163.87118495882785
************************************************
generation:         2
Best_scenario       [44 42 42 42 44 42 42 42]
Best_power          145.3313201778585
************************************************
generation:         3
Best_scenario       [44 42 42 42 44 42 42 42]
Best_power          145.3313201778585
************************************************
generation:         4
Best_scenario       [44 42 42 42 44 42 42 42]
Best_power          145.3313201778585
************************************************
generation:         5
Best_scenario       [44 42 42 42 44 42 42 42]
Best_power          145.3313201778585
************************************************
generation:         6
Best_scenario       [44 42 42 42 42 42 42 42]
Best_power          136.0613877873738
************************************************
generation:         7
Best_scenario       [44 42 42 42 42 42 42 42]
B